# Shared Information

## Workflow

### Agile Methodology

*A brief description of what this actually means and how we are implementing it.*

A large part of this is using Kanbn for detailing, priotising, scheduling, assigning...etc. work ~ Very useful!

### Github Commits
**Gitmoji:** https://gitmoji.dev/
- ```Docs\``` Updates - 📄
- Task (Kanbn) Updates - 📝
- Folder/File Structure Changes - 📂

### Github Branching
- ... (For 2/4/25) How do we not get in the way of each other working best!

# Pipeline

## Azure for Cloud Server

Azure can serve as the cloud infrastructure where your .NET C# project will run. You can leverage various Azure services such as:

- Azure App Services for deploying your application.
- Azure Blob Storage for storing files such as song data, user uploads, etc.
- Azure Kubernetes Service (AKS) if you're scaling with microservices.
- Azure Functions for serverless processing of specific tasks.
- Azure would provide scalability, high availability, and manageability for your application, particularly if it involves cloud-based processing or needs to be accessible from various locations.
  
Where to start:

- Set up an Azure account and explore Azure Portal.
- Try deploying a basic C# .NET Core Web API to Azure App Service.
- Learn about Azure Resource Manager (ARM) templates for infrastructure as code.


## MicroServices Architecture

Microservices would break down your application into smaller, independently deployable services. Each service would handle a specific function, which could make your project more maintainable and scalable. For example:

- A Song Service for managing song metadata and content.
- A User Service for handling user authentication, preferences and progress.

This approach allows each microservice to be developed, deployed, and scaled independently, which makes it easier to improve or replace one part of the system without affecting the rest.

Where to start:

- Build a small .NET Web API service and containerize it with Docker.
- Learn ASP.NET Core minimal APIs for lightweight microservices.
- Set up an API Gateway using Ocelot or Azure API Management.

## Flat file rest API for the songs

**Website:** https://www.geeksforgeeks.org/flat-file-database/

**Flat File:** A flat file can be a binary file or a plain text file such as CSV, txt, or TSV file. 

Where to start:

- Create a simple ASP.NET Core API that reads/writes JSON files.
- Use System.Text.Json or Newtonsoft.Json for serialization.
- Add authentication if needed with JWT or API Keys.

## Agile Methodology for Continous Improvement

- Use Kanban to manage tasks.
- Track progress with JIRA, Azure DevOps, or Trello.
- CI/CD pipelines in GitHub Actions or Azure DevOps for deployment.
- Frequent retrospectives to improve team workflow.

# Setting up

## .NET

- .Net SDK 9.0.201: https://dotnet.microsoft.com/en-us/download/dotnet/9.0

## SQLite

<hr>
Install

```cmd
dotnet add package Microsoft.EntityFrameworkCore.Sqlite
dotnet add package Microsoft.EntityFrameworkCore.Design
```
<hr>

Set up 📂Models/Song.cs
```C#
using System.Collections.Generic;

public class Song
{
    public int Id { get; set; }
    public string Title { get; set; }
    public string Artist { get; set; }
    public string OriginalLyrics { get; set; }
    public string Language { get; set; }
    public List<Translation> Translations { get; set; } = new();
}

public class Translation
{
    public int Id { get; set; }
    public int SongId { get; set; } // Foreign Key
    public Song Song { get; set; }
    public string Type { get; set; } // "word", "sentence", "paragraph"
    public string OriginalText { get; set; }
    public string TranslatedText { get; set; }
    public string TargetLanguage { get; set; }
}
```
* EF Core will automatically create these tables in SQLite.

<hr>

Set up 📂Data/AppDbContext.cs
```C#
using Microsoft.EntityFrameworkCore;

public class AppDbContext : DbContext
{
    public DbSet<Song> Songs { get; set; }
    public DbSet<Translation> Translations { get; set; }

    public AppDbContext(DbContextOptions<AppDbContext> options) : base(options) { }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        if (!optionsBuilder.IsConfigured)
        {
            optionsBuilder.UseSqlite("Data Source=lyrics.db");
        }
    }
}
```

* This tells EF Core to use SQLite and store the database in a file called lyrics.db.
* The database will be created automatically.

<hr>

Open ```LyricalLearning.csproj``` and check SQLite and EF Core tools are referenced.

Register DB in .NET 📂Program.cs

```C#
using Microsoft.EntityFrameworkCore;
using Microsoft.Extensions.DependencyInjection;

var builder = WebApplication.CreateBuilder(args);

// Register SQLite Database
builder.Services.AddDbContext<AppDbContext>(options =>
    options.UseSqlite("Data Source=lyrics.db"));

var app = builder.Build();

app.Run();
```

<hr>

Create DB
```cmd
dotnet ef migrations add InitialCreate
dotnet ef database update
```
* Creates the database file (lyrics.db).
* Generates tables (Songs, Translations).
* Prepares everything for storing lyrics & translations.

<hr>

## Azure

### Creating ASP.NET app & deploying on Azure as a web app

Tutorial Page: https://learn.microsoft.com/en-us/azure/app-service/quickstart-dotnetcore?tabs=netframework48&pivots=development-environment-vscode